In [ ]:
#========================================================================
# Copyright 2019 Science Technology Facilities Council
# Copyright 2019 University of Manchester
#
# This work is part of the Core Imaging Library developed by Science Technology	
# Facilities Council and University of Manchester
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# http://www.apache.org/licenses/LICENSE-2.0.txt
# 
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# 
#=========================================================================

## Tikhonov regularisation using CGLS and block framework
Few lines intro

**Learning objectives:**
1. Construct and manipulate BlockOperators and BlockDataContainer, including direct and adjoint operations and algebra.
2. Use Block Framework to solve Tikhonov regularisation with CGLS algorithm.
3. Apply Tikhonov regularisation to tomographic reconstruction and explain the effect of regularization parameter and operator in regulariser.

In [ ]:
""" 
Tikhonov reconstruction using Block CGLS 
Problem:     min_u alpha * ||\grad u ||^{2}_{2} + || A u - g ||_{2}^{2}
            
             ==> Block CGLS
             
             min_u || \tilde{A} u - \tilde{g} ||_{2}^{2}
             
             \tilde{A} = [ A,
                           alpha * \grad]
             \tilde{g} = [g,
                          0]
             A: Projection operator
             g: Sinogram
"""


from ccpi.framework import ImageGeometry, ImageData, AcquisitionGeometry, BlockDataContainer, AcquisitionData

import numpy as np 
import numpy                          
import matplotlib.pyplot as plt

from ccpi.optimisation.algorithms import CGLS
from ccpi.optimisation.operators import BlockOperator, Gradient, Identity
       
from ccpi.astra.operators import AstraProjectorSimple 
import os

device = "cpu"

# Load  Shepp-Logan phantom 
model = 1 # select a model number from the library
N = 64 # set dimension of the phantom
#path = os.path.dirname(tomophantom.__file__)
#path_library2D = os.path.join(path, "Phantom2DLibrary.dat")
#phantom_2D = TomoP2D.Model(model, N, path_library2D)

ig = ImageGeometry(voxel_num_x = N, voxel_num_y = N)

data = numpy.zeros((N,N))
data = ImageData(geometry=ig)
data = ig.allocate()
x = data.as_array()
x[round(3*N/8):round(5*N/8),round(N/8):round(7*N/8)] = 0.5
x[round(N/8):round(7*N/8),round(3*N/8):round(5*N/8)] = 1.0


detectors = N
angles = np.linspace(0, np.pi, 128, dtype=np.float32)

ag = AcquisitionGeometry('parallel','2D', angles, detectors)

Aop = AstraProjectorSimple(ig, ag, device)    
sin = Aop.direct(data)
noisy_data = AcquisitionData( sin.as_array() + np.random.normal(0,1,ag.shape))
#noisy_data = AcquisitionData( sin.as_array() )

# Show Ground Truth and Noisy Data

fig, (ax1, ax2) = plt.subplots(1, 2,figsize=(10,10))
subplot1 = ax1.imshow(data.as_array())
ax1.set_title('Ground Truth')
subplot2 = ax2.imshow(noisy_data.as_array())
ax2.set_title('Noisy Data')
plt.colorbar(subplot1, ax=ax1,fraction=0.046, pad=0.04)
plt.colorbar(subplot2, ax=ax2,fraction=0.046, pad=0.04)

In [ ]:
# Setup and run the CGLS algorithm  
alpha = 2
Grad = Gradient(ig)
Id = Identity(ig)

In [ ]:
# CGLS simple
x_init = ig.allocate()      
cgls = CGLS(x_init=x_init, operator=Aop, data=noisy_data)
cgls.max_iteration = 1000
cgls.update_objective_interval = 50
cgls.run(1000, verbose = True)

# Show Results and residuals
fig, (ax1, ax2) = plt.subplots(1, 2,figsize=(10,10))
subplot1 = ax1.imshow(cgls.get_output().as_array())
ax1.set_title('CGLS reconstruction')
subplot2 = ax2.imshow(cgls.get_output().as_array() - data.as_array())
ax2.set_title('Residuals')
plt.colorbar(subplot1, ax=ax1,fraction=0.046, pad=0.04)
plt.colorbar(subplot2, ax=ax2,fraction=0.046, pad=0.04)


In [ ]:
# Form Tikhonov as a Block CGLS structure
op_CGLS = BlockOperator( Aop, alpha * Grad, shape=(2,1))
block_data = BlockDataContainer(noisy_data, Grad.range_geometry().allocate())

x_init = ig.allocate()      
cgls = CGLS(x_init=x_init, operator=op_CGLS, data=block_data)
cgls.max_iteration = 1000
cgls.update_objective_interval = 50
cgls.run(1000, verbose = True)

# Show Results and residuals
fig, (ax1, ax2) = plt.subplots(1, 2,figsize=(10,10))
subplot1 = ax1.imshow(cgls.get_output().as_array())
ax1.set_title('CGLS reconstruction')
subplot2 = ax2.imshow(cgls.get_output().as_array() - data.as_array())
ax2.set_title('Residuals')
plt.colorbar(subplot1, ax=ax1,fraction=0.046, pad=0.04)
plt.colorbar(subplot2, ax=ax2,fraction=0.046, pad=0.04)

### Summary